In [201]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import (confusion_matrix, 
                           accuracy_score)

In [49]:
from google.colab import drive
drive.mount('/content/datasets')

Drive already mounted at /content/datasets; to attempt to forcibly remount, call drive.mount("/content/datasets", force_remount=True).


In [202]:
x_path = r'/content/datasets/MyDrive/Datasets/INM-CM4-8 Data for Model Creation/Predictors/All Combined/INMCM4-8_historical_predictors_Point_1.csv'
y_path = r'/content/datasets/MyDrive/Datasets/IMD Rainfall 0.25x0.25/NC Files/csv files/rain_Point_1.csv'
X = pd.read_csv(x_path)
y = pd.read_csv(y_path)

In [203]:
# y1 will hold the rainfall data for logistic regression
y1 = pd.read_csv(y_path)

In [204]:
X.drop(columns=['lon', 'lat', 'Unnamed: 0'], inplace=True)
y.drop(columns=['LONGITUDE', 'LATITUDE'], inplace=True)
y1.drop(columns=['LONGITUDE', 'LATITUDE'], inplace=True)

In [205]:
X.drop_duplicates(keep='first', inplace=True)

In [206]:
X.drop([i for i in range(365)], inplace=True)

In [207]:
y.drop([i for i in range(23376, len(y))], inplace=True)
y1.drop([i for i in range(23376, len(y1))], inplace=True)

In [208]:
y.drop(y[y['TIME'].map(lambda s: s.split()[0][-5:]) == '02-29'].index, inplace = True)
y1.drop(y1[y1['TIME'].map(lambda s: s.split()[0][-5:]) == '02-29'].index, inplace = True)

In [209]:
X.reset_index(inplace=True, drop=True)
y.reset_index(inplace=True, drop=True)
y1.reset_index(inplace=True, drop=True)

In [210]:
X_dtCol = pd.to_datetime(X['time'], format='%Y-%m-%d %H:%M:%S')
X['time'] = X_dtCol

In [211]:
X['day'] = X['time'].dt.day
X['month'] = X['time'].dt.month
X['year'] = X['time'].dt.year
X.drop(columns=['time'], inplace=True)

In [212]:
X

,huss,psl,tas,uas,vas,day,month,year
0,0.013537,100983.783203,296.580307,0.859241,1.870992,1,1,1951
1,0.014403,101124.931641,296.423634,-2.060680,0.271272,2,1,1951
2,0.013132,101207.281738,296.367565,-3.853243,-1.015532,3,1,1951
3,0.013079,101189.250000,296.674225,-4.138630,-2.001815,4,1,1951
4,0.013681,101275.931152,296.668119,-4.534705,-0.849224,5,1,1951
...,...,...,...,...,...,...,...,...
23355,0.011010,100859.258789,295.867292,-0.294562,-2.943822,27,12,2014
23356,0.012659,101134.398926,296.608992,-2.815778,-0.698825,28,12,2014
23357,0.014906,101122.427734,297.672783,-3.172361,-0.537482,29,12,2014
23358,0.016869,101040.858398,297.839441,-3.657494,-0.288831,30,12,2014


In [213]:
Xlin = X

In [214]:
Xlin

,huss,psl,tas,uas,vas,day,month,year
0,0.013537,100983.783203,296.580307,0.859241,1.870992,1,1,1951
1,0.014403,101124.931641,296.423634,-2.060680,0.271272,2,1,1951
2,0.013132,101207.281738,296.367565,-3.853243,-1.015532,3,1,1951
3,0.013079,101189.250000,296.674225,-4.138630,-2.001815,4,1,1951
4,0.013681,101275.931152,296.668119,-4.534705,-0.849224,5,1,1951
...,...,...,...,...,...,...,...,...
23355,0.011010,100859.258789,295.867292,-0.294562,-2.943822,27,12,2014
23356,0.012659,101134.398926,296.608992,-2.815778,-0.698825,28,12,2014
23357,0.014906,101122.427734,297.672783,-3.172361,-0.537482,29,12,2014
23358,0.016869,101040.858398,297.839441,-3.657494,-0.288831,30,12,2014


In [215]:
y.drop(columns=['TIME'], inplace=True)

In [216]:
ylin = y

In [217]:
ylin

,RAINFALL
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
23355,0.000000
23356,0.000000
23357,0.000000
23358,1.345410


In [218]:
y1.drop(columns=['TIME'], inplace=True)

In [ ]:
#y['RAINFALL'].fillna(value=y['RAINFALL'].mean(), inplace=True)
#y1['RAINFALL'].fillna(value=y1['RAINFALL'].mean(), inplace=True)

In [219]:
y1.loc[y1['RAINFALL'] <= 0] = 0
y1.loc[y1['RAINFALL'] > 0] = 1

In [220]:
y1.fillna(0, inplace=True)

In [221]:
y1['RAINFALL'] = y1['RAINFALL'].astype(int)

In [222]:
scaler = StandardScaler()
X = scaler.fit_transform(X[X.columns])
y = scaler.fit_transform(y)

In [223]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.20, random_state=42)

Logistic Regression skearn

In [224]:
model = logisticRegr = LogisticRegression(solver = 'liblinear', C=10.0)
model.fit(X_train, y_train)
predictions = model.predict(X_test)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [225]:
score = logisticRegr.score(X_test, y_test)
print(score)

0.7600599315068494


In [226]:
diditrain = model.predict(X)

In [227]:
diditrain

array([0, 0, 0, ..., 0, 0, 0])

In [228]:
did_rain = pd.DataFrame(data=diditrain)

In [229]:
did_rain = did_rain.rename(columns={0:'RAINFALL'})

In [230]:
did_rain

,RAINFALL
0,0
1,0
2,0
3,0
4,0
...,...
23355,0
23356,0
23357,0
23358,0


In [231]:
did_rain = did_rain[did_rain['RAINFALL']==1]

In [232]:
did_rain

,RAINFALL
174,1
180,1
181,1
182,1
183,1
...,...
23259,1
23261,1
23262,1
23263,1


In [233]:
ylin = ylin[ylin.index.isin(did_rain.index)]

In [234]:
ylin

,RAINFALL
174,8.820212
180,0.000000
181,0.000000
182,0.000000
183,0.000000
...,...
23259,0.000000
23261,30.615278
23262,0.000000
23263,1.057206


In [236]:
Xlin = Xlin[Xlin.index.isin(did_rain.index)]

In [237]:
Xlin

,huss,psl,tas,uas,vas,day,month,year
174,0.021299,100100.508789,301.491533,2.601320,4.061915,24,6,1951
180,0.019680,100019.802734,299.649563,3.869478,4.628991,30,6,1951
181,0.019548,100112.823242,299.507870,4.587652,4.060770,1,7,1951
182,0.019766,100148.586914,299.903727,4.558264,3.726631,2,7,1951
183,0.020052,100150.206543,299.767683,4.451090,3.795679,3,7,1951
...,...,...,...,...,...,...,...,...
23259,0.018443,100364.908691,300.322104,2.775795,1.046591,22,9,2014
23261,0.019776,100563.153809,300.187410,1.696842,3.272416,24,9,2014
23262,0.019960,100503.102051,300.260748,2.852921,2.996651,25,9,2014
23263,0.020072,100405.176758,300.359875,3.241900,2.098283,26,9,2014


In [238]:
ylin['RAINFALL'].fillna(value=ylin['RAINFALL'].mean(), inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Logistic Regression statsmodel.api

In [22]:
log_reg = sm.Logit(y_train, X_train).fit()

Optimization terminated successfully.
         Current function value: 0.599806
         Iterations 5


In [23]:
print(log_reg.summary())

                           Logit Regression Results                           
Dep. Variable:               RAINFALL   No. Observations:                18688
Model:                          Logit   Df Residuals:                    18680
Method:                           MLE   Df Model:                            7
Date:                Fri, 30 Apr 2021   Pseudo R-squ.:                -0.01891
Time:                        12:16:08   Log-Likelihood:                -11209.
converged:                       True   LL-Null:                       -11001.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.5376      0.033     16.340      0.000       0.473       0.602
x2            -0.0546      0.044     -1.238      0.216      -0.141       0.032
x3            -0.3064      0.031     -9.755      0.0

In [24]:
yhat = log_reg.predict(X_test)
prediction = list(map(round, yhat))

In [25]:
cm = confusion_matrix(y_test, prediction) 
print ("Confusion Matrix : \n", cm) 
  
# accuracy score of the model
print('Test accuracy = ', accuracy_score(y_test, prediction))

Confusion Matrix : 
 [[2212 1156]
 [ 245 1059]]
Test accuracy =  0.7001284246575342


LINEAR REGRESSION IF IT RAINS

In [239]:
scaler = StandardScaler()
Xlin = scaler.fit_transform(Xlin[Xlin.columns])

In [240]:
ylin = scaler.fit_transform(ylin)

In [245]:
X_train, X_test, y_train, y_test = train_test_split(Xlin, ylin, test_size=0.33, random_state=42)

In [246]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [247]:
predictions = np.where(predictions<=0, 0, predictions)

In [248]:
print('Point6: ')
print(f'The Root Mean Square error is: {np.sqrt(mean_squared_error(y_test, predictions))}')
print(f'The Mean Absolute error is: {mean_absolute_error(y_test, predictions)}')
print(f'The R2 Score is: {r2_score(y_test, predictions)}')

Point6: 
The Root Mean Square error is: 0.9100508326255348
The Mean Absolute error is: 0.5955258946653961
The R2 Score is: -0.005745774396808123
